In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Este repositório trata-se de uma análise do volume de vendas de uma rede de supermercados na semanas que precedem alguns feriados nacionais ##

Os dados para analise correspondem a dados históricos de vendas de 45 lojas Walmart localizadas em diferentes regiões. Cada loja contém vários departamentos

O desafio consiste em prever as vendas em todo o departamento para cada loja por meio de uma modelagem que consiga identificar os efeitos de remarcações nessas semanas de férias na ausência de dados históricos completos / ideais.

<h1> Índice Analítico </h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#Carregando_Conhecendo o dataset">Carregando e conhecendo o dataset</a></li>
        <li><a href="#pre-processessamento">pre-processessamento</a></li>
        <li><a href="#Visualização previa">Visualização prévia</a></li>
        <li><a href="#Inferencias previas">Inferências prévias</a></li>
        <li><a href="#setting_up_tree">Configurando a Árvore de Decisão</a></li>
        <li><a href="#modeling">Modelagem</a></li>
        <li><a href="#prediction">Predição</a></li>
        <li><a href="#evaluation">Avaliação</a></li>
        <li><a href="#visualization">Visualização Final</a></li>
    </ol>
</div>
<br>
<hr>

Importe as bibliotecas necessárias para a analise

In [ ]:
## Importe as bilbiotecas##
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
import os
import seaborn as sns # visualization
from scipy import stats
from scipy.stats import norm 
import warnings 
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings('ignore') #ignore warnings

%matplotlib inline
import gc

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
pwd

<div id="Carregando_Conhecendo o dataset"> 
    <h2>Carregando e conhecendo o dataset</h2>
    
Os arquivos podem ser encontrados em https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data
e correspondem: 

#stores.csv#
Este arquivo contém informações anônimas sobre as 45 lojas, indicando o tipo e tamanho da loja.

#train.csv#
Esses são os dados históricos do treinamento, que abrangem desde 05-02-2010 a 01/11/2012. Dentro deste arquivo, você encontrará os seguintes campos:

Loja - o número da loja
Departamento - o número do departamento
Data - a semana
Weekly_Sales - vendas para o departamento especificado na loja especificada
IsHoliday - se a semana é uma semana especial de férias

#test.csv#
Este arquivo é idêntico ao train.csv, exceto que retivemos as vendas semanais. Você deve prever as vendas para cada trigêmeo de loja, departamento e data neste arquivo.

#features.csv#
Este arquivo contém dados adicionais relacionados à loja, departamento e atividade regional para as datas especificadas. Ele contém os seguintes campos:

Loja - o número da loja
Data - a semana
Temperatura - temperatura média na região
Fuel_Price - custo do combustível na região
MarkDown1-5 - dados anonimizados relacionados a descontos promocionais que o Walmart está executando. Os dados do MarkDown estão disponíveis somente após novembro de 2011 e não estão disponíveis para todas as lojas o tempo todo. Qualquer valor ausente é marcado com um NA.
CPI - o índice de preços ao consumidor
Desemprego - a taxa de desemprego
IsHoliday - se a semana é uma semana especial de férias
Por conveniência, os quatro feriados se enquadram nas semanas seguintes no conjunto de dados (nem todos os feriados estão nos dados):

Super Bowl: 12-fev-10, 11-fev-11, 10-fev-12, 8-fev-13
Dia do Trabalhador: 10-set-10, 9-set-11, 7-set-12, 6-set -13
Ação de Graças: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
Natal: 31-Dez-10, 30-Dez-11, 28-Dez-12, 27-Dez -13

Lendo os dados usando o pandas dataframe:

In [ ]:
tt=pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
tst=pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip", parse_dates=["Date"])
lj=pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")
fat = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])

Trabalharemos com 'train.csv', 'test.csv', 'store.csv' e para análises externas usaremos 'features.csv' 

<div id="pre-processessamento"> 
    <h2>Pre-Processessamento</h2>

In [ ]:
#viasualiza previamente 5 linha do arquivo 'features.csv'
tt.head (5)

Converte Data em objeto

In [ ]:
#Converte date to object do arquivo 'features.csv'# 
fat['Date'] = pd.to_datetime(fat['Date']) #indica o titulo da coluna que quer converter 
fat.head(2)

In [ ]:
#viasualiza previamente 5 linha do arquivo 'test.csv'
tst.head (5)

In [ ]:
#Converte date to object do arquivo 'test.csv'# 
tst['Date'] = pd.to_datetime(tst['Date']) #indica o titulo da coluna que quer converter 
tst.head(2)

In [ ]:
#viasualiza previamente 5 linha do arquivo 'store.csv'
lj.head (5)

In [ ]:
#viasualiza previamente 5 linha do arquivo 'train.csv'
tt.head (5)

In [ ]:
#Converte date to object do arquivo 'train.csv'# 
tt['Date'] = pd.to_datetime(tt['Date']) #indica o titulo da coluna que quer converter 
tt.head(2)

Vamos ver quantas de cada classe há em nosso conjunto de dados

In [ ]:
 fat['IsHoliday'] . value_counts ()

No arquivo 'features.csv'

temos 7605 datas e eventos globais que não correspondem a feriados 

e 585 eventos globais que correspondem a feriados

In [ ]:
tt['IsHoliday'] . value_counts ()

No arquivo 'train.csv'

temos 391909 datas e faturamentos não corresponder a feriados 
e 29661 datas e eventos correspondem a feriados

In [ ]:
print("A estrutura dos dados de treinamento é:", tt.shape)
print("A estrutura dos dados de treinamento é:", tst.shape)
print("A proporção entre os dados de treinamento e os dados de teste é:", (round(tt.shape[0]*100/(tt.shape[0]+tst.shape[0])),100-round(tt.shape[0]*100/(tt.shape[0]+tst.shape[0]))))

In [ ]:
tt=tt.merge(lj, on='Store', how='left')
tt.head()

Subdividimos a coluna Date em ano/mês/semana/dia

In [ ]:
tt['Ano']=tt['Date'].dt.year
tt['Mês']=tt['Date'].dt.month
tt['Semana']=tt['Date'].dt.week
tt['Dia']=tt['Date'].dt.day
tt['n_dias']=(tt['Date'].dt.date-tt['Date'].dt.date.min()).apply(lambda x:x.days)

In [ ]:
Ano=pd.Series(tt['Ano'].unique())
Semana=pd.Series(tt['Semana'].unique())
Mês=pd.Series(tt['Mês'].unique())
Dia=pd.Series(tt['Dia'].unique())
n_dias=pd.Series(tt['n_dias'].unique())

In [ ]:
tst['Ano']=tst['Date'].dt.year
tst['Mês']=tst['Date'].dt.month
tst['Semana']=tst['Date'].dt.week
tst['Dia']=tst['Date'].dt.day
tst['n_dias']=(tst['Date'].dt.date-tst['Date'].dt.date.min()).apply(lambda x:x.days)

In [ ]:
Ano=pd.Series(tst['Ano'].unique())
Semana=pd.Series(tst['Semana'].unique())
Mês=pd.Series(tst['Mês'].unique())
Dia=pd.Series(tst['Dia'].unique())
n_dias=pd.Series(tst['n_dias'].unique())

In [ ]:
print("O formato do conjunto de dados 'store.csv' é: ", lj.shape)
print("Os numeros das lojas são: ", lj['Store'].unique())
print("Os tipos de lojas são:", lj['Type'].unique())

#### Verificar influência de fatores externos ao volume de vendas das lojas

inicialmente faremos um pré-processamento dos dados

In [ ]:
fat['Ano']=fat['Date'].dt.year
fat['Mês']=fat['Date'].dt.month
fat['Semana']=fat['Date'].dt.week
fat['Dia']=fat['Date'].dt.day
#tt['n_dias']=(tt['Date'].dt.date-tt['Date'].dt.date.min()).apply(lambda x:x.days)

In [ ]:
Ano=pd.Series(fat['Ano'].unique())
Semana=pd.Series(fat['Semana'].unique())
Mês=pd.Series(fat['Mês'].unique())
Dia=pd.Series(fat['Dia'].unique())
#n_dias=pd.Series(fat['n_dias'].unique())

In [ ]:
fat.head()

A base de dados faetures esta composta com treinamento e teste.

Portanto para realizar uma análise comparativa entre os dados de treinamento (tt) e as influências externas (fat). É necessário eliminar do conjunto (fat) as linhas posteriores a 10/2012

Criamos um varável que receberá as linhas que queremos eliminar de (fat)

In [ ]:
fat_1 = fat.loc[(fat['Ano'] > 2012)
                |(fat['Mês'] > 10) & (fat['Ano']== 2012)]
tfat = fat.drop(fat_1.index)
tfat.head()

In [ ]:
tfat.shape

In [ ]:
fatext = ['Store','Date', 'Temperature','Fuel_Price','CPI','Unemployment','IsHoliday']
fat_tt = tfat.filter(items=fatext)
fat_tt.head()

In [ ]:
fatext = ['Store','Date', 'Temperature','Fuel_Price','CPI','Unemployment','IsHoliday']
fat_tst = fat_1.filter(items=fatext)
fat_tst.head()

In [ ]:
fat_tt.shape

In [ ]:
fatext = ['Store','Date', 'Temperature','Fuel_Price','CPI','Unemployment','IsHoliday']
fat_2 = tfat.filter(items=fatext)
fat_2.head()

In [ ]:
fat_tst.shape

In [ ]:
tfat = tt.groupby(['Store','Date']).agg({'Weekly_Sales': np.mean})
tfat.index_col=0
tfat.head()

In [ ]:
tst=tst.merge(lj, on='Store', how='left')
tst.head()


In [ ]:
tstfat = tst.groupby(['Store','Date']).agg({'Size':np.mean})
tstfat.index_col=0
tstfat.head()

In [ ]:
tstfat.shape

In [ ]:
tt_1 = tfat.reset_index(level=['Store', 'Date'])
tst_1 = tstfat.reset_index(level=['Store', 'Date'])
tt_1.head()

In [ ]:
tst_1.head()

In [ ]:
tt_1['Date'] = pd.to_datetime(tt_1['Date'])
tst_1['Date'] = pd.to_datetime(tst_1['Date'])

Apos as alterações necessárias, as bases de dados sobre:
    
    - Influências externas treinamento recebe o nome de  ( fat_tt );
    - Base de dados de treinamento o nome de  ( tt_1 );
Que, após serem acopladas recebe o nome nome  ( New_fat ) 

In [ ]:
New_fat=pd.merge(fat_tt, tt_1,
                      on=['Date', 'Store'])
New_fat.head()

Apos as alterações necessárias, as bases de dados sobre:
    
    - Influências externas recebe o nome de  ( fat_tst );
    - Base de dados de treinamento o nome de  ( tst_1 );
Que, após serem acopladas recebe o nome nome  ( New_fat_tst ) 

In [ ]:
New_fat_tst=pd.merge(fat_tst, tst_1,
                      on=['Date', 'Store'])

In [ ]:
New_fat_tst.head()

In [ ]:
print(New_fat.describe()['Weekly_Sales'].round(2))

#### Uma breve análise por tipo de loja ####

In [ ]:
print(lj.head())
grouped=lj.groupby('Type')
print(grouped.describe()['Size'].round(2))

#### Uma breve análise dos efeitos externos nas vendas

In [ ]:
print(tt_1.head())
grouped=tt_1.groupby('Store')
print(grouped.describe()['Weekly_Sales'].round(2))

<div id="Visualização previa"> 
    <h2>Visualização prévia e Análise da relação entre variáveis</h2>

Análise visual por tipos de lojas e o tamanho das lojas

In [ ]:
data = pd.concat([lj['Type'], lj['Size']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))

plt.xlabel('Tipo')
plt.ylabel('Tamanho da loja')

fig = sns.boxplot(x='Type', y='Size', data=data)

In [ ]:
data = pd.concat([tt['Type'], tt['Weekly_Sales']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
plt.xlabel('Tipo')
plt.ylabel('Venda semanal')
fig = sns.boxplot(x='Type', y='Weekly_Sales', data=data, showfliers=False)

As lojas do tipo A são maiores e as do tipo C são menores.
Podemos observar que a mediana do volume semanal de venda das lojas tipo A é a mais alta e do tipo C é a mais baixa.
Isso significa que lojas maiores têm maior volume de vendas.

In [ ]:
plt.style.use('ggplot')

fig=plt.figure()
ax=fig.add_subplot(111)
plt.xlabel('Tamanho da loja')
plt.ylabel('Venda semanal')
ax.scatter(tt['Size'],tt['Weekly_Sales'], alpha=0.5)

plt.show()

In [ ]:
types=lj['Type'].unique()

plt.style.use('ggplot')

fig=plt.figure(figsize=(10,5))
ax=fig.add_subplot(111)

for t in types:
    x=tt.loc[tt['Type']==t, 'Size']
    y=tt.loc[tt['Type']==t, 'Weekly_Sales']
    
    ax.scatter(x,y,alpha=0.5, label=t)

ax.set_title('Gráfico de dispersão do tamanho da loja volume de venda e tipo de loja')
ax.set_xlabel('Tamanho')
ax.set_ylabel('Venda semanal')

ax.legend(loc='higher right',fontsize=12)
plt.style.use('classic')

plt.show()

Embora o resultado não aperente um destinçao clara entre o Tipo de loja, tamanho e volume de vendas semanais.
<br>
É possivel observar um aumento no volume de vendas nas lojas do tipo A e B 
<br>
<br>
O gráfico a seguir podemos observar o volume de venda de cada loja, bem como o tipo (A,B e C) de cada loja 

In [ ]:
data = pd.concat([tt['Store'], tt['Weekly_Sales'], tt['Type']], axis=1)
f, ax = plt.subplots(figsize=(20, 8))
fig = sns.boxplot(x='Store', y='Weekly_Sales', data=data, showfliers=False, hue="Type")

O gráfico confirma o maior volume de vendas ente as lojas do tipo A

In [ ]:
####### alterar no dataframe Store (str) to Store (int) 
# mod= 'Weekly_Sales ~ C(Store) + C(Type)'
# lm= ols(mod, data=data).fit()
# aov_tab = sm.stats.anova_lm(mod, type=2)
# print (aov_tab)

In [ ]:
New_fat.head()

In [ ]:
##############################     Verificar (gráfico ruim)
data = pd.concat([New_fat['Temperature'], New_fat['Weekly_Sales'], New_fat['IsHoliday']], axis=1)
f, ax = plt.subplots(figsize=(30, 8))
plt.title('Relação entre Desemprego e vendas') 
fig = sns.boxplot(x='Temperature', y='Weekly_Sales', data=data, showfliers=False, hue="IsHoliday")

In [ ]:
data = pd.concat([New_fat['Fuel_Price'], New_fat['Weekly_Sales'], New_fat['IsHoliday']], axis=1)
f, ax = plt.subplots(figsize=(25, 8))
plt.title('Relação entre Preço da gasolina e vendas') 
fig = sns.boxplot(x='Fuel_Price', y='Weekly_Sales', data=data, showfliers=False, hue="IsHoliday")

In [ ]:
data = pd.concat([New_fat['Unemployment'], New_fat['Weekly_Sales'], New_fat['IsHoliday']], axis=1)
f, ax = plt.subplots(figsize=(27, 8))
plt.title('Relação entre Desemprego e vendas') 
fig = sns.boxplot(x='Unemployment', y='Weekly_Sales', data=data, showfliers=False, hue="IsHoliday")

A loja pode ser uma das variáveis que fornece informações sobre vendas. Porem as lojas fornecem muitas informações intrisecas, como tipo, tamanho e departamento. 
Observamos tambem que as variáveis Preço do combustivel (Fuel_Price) e desemprego (Unemployment) fornecem informações sobre o volume de venda. 
Ja os dados referentes a temperatura não podemos observar que fornece informações sobre o volume de venda semanal 

In [ ]:
def plota_bar_dupla_1():
   grupos = 45
   HolidayTrue = (tt['IsHoliday==True'])
   HolidayFalse = (tt['IsHoliday==False'])
   fig, ax = plt.subplots()
   indice = np.arange(grupos)
   bar_larg = 0.4
   transp = 0.7
   plt.bar(indice, HolidayTrue, bar_larg, alpha=transp, color=azul, label='Holiday')
   plt.bar(indice + bar_larg, HolidayFalse, bar_larg, alpha=transp, color=verde, label='No Holiday')

plt.xlabel('Weekly_Sales') 
plt.ylabel('Stores') 
plt.title('Vendas por lojas') 
#plt.xticks(indice + bar_larg) 
plt.legend() 
plt.tight_layout() 
plt.show()

In [ ]:
#Tentar colocar verdadeiro e falso para feriado ao lado e todas as lojas no eixo x
bins = np.linspace(tt.Weekly_Sales.min(), tt.Weekly_Sales.max(), 10)
g = sns.FacetGrid(tt, col="Store", hue="IsHoliday", palette="Set1", col_wrap=5)
g.map(plt.hist, 'Weekly_Sales', bins=bins, ec="k")

plt.title('Volume máximo e minimo das Vendas por lojas') 
g.axes[-1].legend()
plt.show()

Podemos observar que o volume máximo de vendas ocorre em não feriados 

In [ ]:
##Correlaçao
def plot_corr(corr):
    # Cortaremos a metade de cima pois é o espelho da metade de baixo
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask, 1)] = True

    sns.heatmap(corr, mask=mask, cmap='RdBu', square=True, linewidths=.2)

# Calculando a correlação
corr = tt.corr() 
plt.title('Correlação entre os dados') 
plot_corr(corr)

A correlação entre os dados nao nos apresenta respostas relevantes. Pode-se observar uma pequena relevância entre a o volume de vendas da loja e seu tamanho. Porem não ha indicações relevantes.

In [ ]:
data = pd.concat([tt['Store'], tt['Weekly_Sales'], tt['IsHoliday']], axis=1)
f, ax = plt.subplots(figsize=(30, 8))
fig = sns.boxplot(x='Store', y='Weekly_Sales', data=data, showfliers=False, hue="IsHoliday")
plt.title('Relação entre as lojas, vendas semanais e feriados') 

Feriados e lojas não mostram relações significativas.

In [ ]:
data = pd.concat([tt['Dept'], tt['Weekly_Sales'], tt['Type']], axis=1)
f, ax = plt.subplots(figsize=(25, 10))
plt.title('Relação entre as Departamento e vendas semanais') 
fig = sns.boxplot(x='Dept', y='Weekly_Sales', data=data, showfliers=False)

Observamos maior volume de venda nas lojas 2, 38, 40, 72, 90 - 95. 

In [ ]:
data = pd.concat([tt['Dept'], tt['Weekly_Sales'], tt['Type']], axis=1)
f, ax = plt.subplots(figsize=(10, 50))
fig = sns.boxplot(y='Dept', x='Weekly_Sales', data=data, showfliers=False, hue="Type",orient="h") 

Cada departamento mostra os diferentes níveis de vendas, podemos inferir que há diferença nas vendas 
Departamento pode ser a variável poderosa para prever vendas
Quando departamento e tipo de loja são considerados juntos, geralmente o departamento do tipo A mostra o maior recorde de vendas
Suposição 4: Tipo e departamento podem ter o efeito de interação

In [ ]:
data = pd.concat([tt['Dept'], tt['Weekly_Sales'], tt['IsHoliday']], axis=1)
f, ax = plt.subplots(figsize=(25, 10))
fig = sns.boxplot(x='Dept', y='Weekly_Sales', data=data, showfliers=False, hue="IsHoliday")

Diferente da relação de loja e feriado, departamento e feriado não explicam nenhuma relação. 
 O departamento 72 mostra o maior aumento nas vendas durante os feriados 
No entanto, outros não, e ainda mais em alguns departamentos, as vendas fora dos feriados são maiores.
Isso significa que o caráter do produto (departamento) é diferente da relação com as vendas


In [ ]:
plt.style.use('ggplot')
fig, axes = plt.subplots(1,2, figsize = (20,5))
fig.subplots_adjust(wspace=1, hspace=1)
fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

sales_holiday=tt[['IsHoliday','Weekly_Sales']]
target=[sales_holiday['Weekly_Sales'].loc[sales_holiday['IsHoliday']==True],sales_holiday['Weekly_Sales'].loc[sales_holiday['IsHoliday']==False]]
labels=['Holiday','Not Holiday']

#median
medianprop={'color':'#2196F3',
            'linewidth': 2,
            'linestyle':'-'}

# outliers

flierprop={'color' : '#EC407A',
          'marker' : 'o',
          'markerfacecolor': '#2196F3',
          'markeredgecolor':'white',
          'markersize' : 3,
          'linestyle' : 'None',
          'linewidth' : 0.1}



axes[0].boxplot(target,labels=labels, patch_artist = 'Patch',
                  showmeans=True,
                  flierprops=flierprop,
                  medianprops=medianprop)

plt.title('Volume de vendas') 


axes[1].boxplot(target,labels=labels, patch_artist = 'Patch',
                  showmeans=True,
                  flierprops=flierprop,
                  medianprops=medianprop)

axes[1].set_ylim(-6000,80000)

plt.show()

######################## Explicar melhor esse gggrafico #################################

In [ ]:
print(tt[tt['IsHoliday']==True]['Weekly_Sales'].describe().round(1))
print(tt[tt['IsHoliday']==False]['Weekly_Sales'].describe().round(1))

Vendas em feriados são um pouco mais do que vendas em não feriados

In [ ]:
data = pd.concat([tt['Mês'], tt['Weekly_Sales']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='Mês', y="Weekly_Sales", data=data, showfliers=False)
plt.title('Relação vendas semanais por mes') 

In [ ]:
data = pd.concat([tt['Mês'], tt['Weekly_Sales'],tt['IsHoliday']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='Mês', y="Weekly_Sales", data=data, showfliers=False, hue='IsHoliday')
plt.title('Relação entre vendas semanais, meses e feriados') 

In [ ]:
data = pd.concat([tt['Mês'], tt['Weekly_Sales'],tt['Type']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='Mês', y="Weekly_Sales", data=data, showfliers=False, hue='Type')
plt.title('Relação entre vendas semanais, meses e tipos de lojas')

In [ ]:
data = pd.concat([tt['Ano'], tt['Weekly_Sales']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='Ano', y="Weekly_Sales", data=data, showfliers=False)
plt.title('Relação entre vendas semanais e Anos')

In [ ]:
data = pd.concat([tt['Semana'], tt['Weekly_Sales']], axis=1)
f, ax = plt.subplots(figsize=(20, 6))
fig = sns.boxplot(x='Semana', y="Weekly_Sales", data=data, showfliers=False)
plt.title('Relação entre vendas semanais, Semanas')

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(tt['Weekly_Sales'])
plt.title('Distribuição de vendas semanais')

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
grouped=tt.groupby(['Dept','Weekly_Sales','IsHoliday']).mean().round(0).reset_index()
sns.distplot(grouped)
plt.title('Distribuição de vendas semanais por departamentto no feriado')

In [ ]:
print("Skewness: ", tt['Weekly_Sales'].skew()) #skewness
print("Kurtosis: ", tt['Weekly_Sales'].kurt()) #kurtosis
tt['Weekly_Sales'].min()

In [ ]:
fig.add_subplot(1,2,1)
res = stats.probplot(tt.loc[tt['Weekly_Sales']>0,'Weekly_Sales'], plot=plt)

fig.add_subplot(1,2,2)
res = stats.probplot(np.log1p(tt.loc[tt['Weekly_Sales']>0,'Weekly_Sales']), plot=plt)

In [ ]:
tt.describe()['Weekly_Sales']

In [ ]:
tt_over_zero=tt[tt['Weekly_Sales']>0]
tt_below_zero=tt[tt['Weekly_Sales']<=0]
sales_over_zero = np.log1p(tt_over_zero['Weekly_Sales'])
#histogram
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(sales_over_zero)

In [ ]:
print("Skewness: ", sales_over_zero.skew()) #skewness
print("Kurtosis: ", sales_over_zero.kurt()) #kurtosis

In [ ]:
grouped=tt.groupby(['Dept','Date']).mean().round(0).reset_index()
print(grouped.shape)
print(grouped.head())
data=grouped[['Dept','Date','Weekly_Sales']]


dept=tt['Dept'].unique()
dept.sort()
dept_1=dept[0:20]
dept_2=dept[20:40]
dept_3=dept[40:60]
dept_4=dept[60:]

fig, ax = plt.subplots(2,2,figsize=(20,10))
fig.subplots_adjust(wspace=0.5, hspace=0.5)
fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

for i in dept_1 :
    data_1=data[data['Dept']==i]
    ax[0,0].plot(data_1['Date'], data_1['Weekly_Sales'],label='Dept_1_mean_sales')

for i in dept_2 :
    data_1=data[data['Dept']==i]
    ax[0,1].plot(data_1['Date'], data_1['Weekly_Sales'],label='Dept_1_mean_sales')
    
for i in dept_3 :
    data_1=data[data['Dept']==i]
    ax[1,0].plot(data_1['Date'], data_1['Weekly_Sales'],label='Dept_1_mean_sales')    

for i in dept_4 :
    data_1=data[data['Dept']==i]
    ax[1,1].plot(data_1['Date'], data_1['Weekly_Sales'],label='Dept_1_mean_sales')        
    
ax[0,0].set_title('Mean sales record by department(0~19)')
ax[0,1].set_title('Mean sales record by department(20~39)')
ax[1,0].set_title('Mean sales record by department(40~59)')
ax[1,1].set_title('Mean sales record by department(60~)')


ax[0,0].set_ylabel('Mean sales')
ax[0,0].set_xlabel('Date')
ax[0,1].set_ylabel('Mean sales')
ax[0,1].set_xlabel('Date')
ax[1,0].set_ylabel('Mean sales')
ax[1,0].set_xlabel('Date')
ax[1,1].set_ylabel('Mean sales')
ax[1,1].set_xlabel('Date')


plt.show()

Pelo gráfico de linhas, podemos ver os seguintes

O nível de vendas é diferente por departamento e o nível de registro de vendas de um departamento é estável
Há alguns pontos altos em janeiro e maio. Portanto, pode haver um evento para altas vendas
Alguns departamentos estão altamente relacionados com esses eventos. Assim, o recorde de vendas sobe acentuadamente em torno de janeiro ou maio


In [ ]:
grouped=tt.groupby(['Store','Date']).mean().round(0).reset_index()
grouped.shape
grouped.head()

data=grouped[['Store','Date','Weekly_Sales']]
type(data)


store=tt['Store'].unique()
store.sort()
store_1=store[0:5]
store_2=store[5:10]
store_3=store[10:15]
store_4=store[15:20]
store_5=store[20:25]
store_6=store[25:30]
store_7=store[30:35]
store_8=store[35:40]
store_9=store[40:]

fig, ax = plt.subplots(5,2,figsize=(20,15))

fig.subplots_adjust(wspace=0.5, hspace=0.5)
fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

for i in store_1 :
    data_1=data[data['Store']==i]
    ax[0,0].plot(data_1['Date'], data_1['Weekly_Sales'])
    
for i in store_2 :
    data_2=data[data['Store']==i]
    ax[0,1].plot(data_2['Date'], data_2['Weekly_Sales'])
    
for i in store_3 :
    data_3=data[data['Store']==i]
    ax[1,0].plot(data_3['Date'], data_3['Weekly_Sales'])

for i in store_4 :
    data_4=data[data['Store']==i]
    ax[1,1].plot(data_4['Date'], data_4['Weekly_Sales'])
    
for i in store_5 :
    data_5=data[data['Store']==i]
    ax[2,0].plot(data_5['Date'], data_5['Weekly_Sales'])  

for i in store_6 :
    data_6=data[data['Store']==i]
    ax[2,1].plot(data_6['Date'], data_6['Weekly_Sales'])  

for i in store_7 :
    data_7=data[data['Store']==i]
    ax[3,0].plot(data_7['Date'], data_7['Weekly_Sales'])      

for i in store_8 :
    data_8=data[data['Store']==i]
    ax[3,1].plot(data_8['Date'], data_8['Weekly_Sales'])     
    
for i in store_9 :
    data_9=data[data['Store']==i]
    ax[4,0].plot(data_9['Date'], data_9['Weekly_Sales'])     

    
ax[0,0].set_title('Mean sales record by store(0~4)')
ax[0,1].set_title('Mean sales record by store(5~9)')
ax[1,0].set_title('Mean sales record by store(10~14)')
ax[1,1].set_title('Mean sales record by store(15~19)')
ax[2,0].set_title('Mean sales record by store(20~24)')
ax[2,1].set_title('Mean sales record by store(25~29)')
ax[3,0].set_title('Mean sales record by store(30~34)')
ax[3,1].set_title('Mean sales record by store(35~39)')
ax[4,0].set_title('Mean sales record by store(40~)')



ax[0,0].set_ylabel('Mean sales')
ax[0,0].set_xlabel('Date')
ax[0,1].set_ylabel('Mean sales')
ax[0,1].set_xlabel('Date')
ax[1,0].set_ylabel('Mean sales')
ax[1,0].set_xlabel('Date')
ax[1,1].set_ylabel('Mean sales')
ax[1,1].set_xlabel('Date')
ax[2,0].set_ylabel('Mean sales')
ax[2,0].set_xlabel('Date')
ax[2,1].set_ylabel('Mean sales')
ax[2,1].set_xlabel('Date')
ax[3,0].set_ylabel('Mean sales')
ax[3,0].set_xlabel('Date')
ax[3,1].set_ylabel('Mean sales')
ax[3,1].set_xlabel('Date')
ax[4,0].set_ylabel('Mean sales')
ax[4,0].set_xlabel('Date')



plt.show()

Suposição: o maior dia de vendas (por exemplo, Natal) fornecerá alto poder de previsão
Suposição: o dia mais alto de vendas será diferente por departamento e loja (por exemplo, alguns departamentos não são sensíveis ao Natal)
Assim, extraia o dia mais alto e combine esse dia com o conjunto de dados de treinamento

In [ ]:
grouped=tt.groupby(['Store','Dept'])['Weekly_Sales'].max().reset_index()
grouped['Store']=grouped['Store'].astype(str)
grouped['Dept']=grouped['Dept'].astype(str)
grouped['Weekly_Sales']=grouped['Weekly_Sales'].astype(str)
grouped['key']=grouped['Weekly_Sales'] +'_'+ grouped['Store'] +'_'+ grouped['Dept']


tt['Store']=tt['Store'].astype(str)
tt['Dept']=tt['Dept'].astype(str)
tt['Weekly_Sales_2']=tt['Weekly_Sales'].astype(str)
tt['key']=tt['Weekly_Sales'].astype(str) +'_'+ tt['Store'].astype(str) +'_'+ tt['Dept'].astype(str)

train_2=pd.merge(tt, grouped['key'], how='inner', on='key' )
train_2['Date_2']=train_2['Mês'].astype(str) + '-' + train_2['Dia'].astype(str)

grouped_2=train_2.groupby(['Date_2','Store','Dept']).count().reset_index()
grouped_2.sort_values('Weekly_Sales',ascending=False,inplace=True)

In [ ]:
grouped_2['key_2']=grouped_2['Date_2'].astype(str) + grouped_2['Store'].astype(str) + grouped_2['Dept'].astype(str)
grouped_2['Count']=grouped_2['Weekly_Sales']
data=grouped_2[['key_2','Count']]

tt['Date_2']=tt['Mês'].astype(str) + '-' + tt['Dia'].astype(str)
tt['key_2']=tt['Date_2'].astype(str) + tt['Store'].astype(str) + tt['Dept'].astype(str)
train=pd.merge(tt, data, how='left', on='key_2' )
train.loc[train['Count'].isnull(),'Count']=0

#grouped_2['proportion']=grouped_2['Weekly_Sales']/sum(grouped_2['Store'])
#grouped_2['Count']=grouped_2['Weekly_Sales']
#data=grouped_2[['Date_2','Count']]
#print(data.head(100))

#train['Date_2']=train['Month'].astype(str) + '-' + train['Day'].astype(str)

#train=pd.merge(train, data, how='left', on='Date_2' )
#train.head(150)

In [ ]:
grouped.head()

In [ ]:
train_2.head()

In [ ]:
grouped_2.head()

In [ ]:
train_2.head()

Após realizar o agrupamento entre a janla de datas o departemaneto e a loja (incluindo o tipo)
Iremos plotar a contagem destes elementos associacodos a as vendas semanais. 

In [ ]:
data = pd.concat([train['Count'], train['Weekly_Sales'], train['Store']], axis=1)
f, ax = plt.subplots(figsize=(5, 5))
fig=sns.boxplot(x='Count', y="Weekly_Sales", data=data, showfliers=False)

Count 0 indica a associação do dia com venda de maxima, departamento com venda máxima e loja com niveis máximos de vendas, em não feirados 

Ja count 1 indica  a associação do dia com venda de maxima, departamento com venda máxima e loja com niveis máximos de vendas volume de vendas em feriados.

Deste modo podemos obseervar volume máx de vendas este acumudado em feriados para dias especificos e departamentos especificos em lojas especificas.

Esse volume aparenta estar associado ao periodo Natalino. 
                 
                 
 

# Machine Learning #

SVM Para Classificação vendas por feirado
indicando se é feriado ou não pelo volume de vendas das lojas Se é feriado ou não pelo volume vendas por departamento

Preparando dados 

In [ ]:
from sklearn import svm 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
tt.data = tt[['Store','Dept','Weekly_Sales','Size','Ano','Mês','Semana','Dia','n_dias']]
tt.targed = tt[['IsHoliday']]
tst.data = tst[['Store', 'Dept', 'Type','Size','Ano','Mês','Semana','Dia','n_dias']]
tst.targed = tst[['IsHoliday']]
#Convertendo features (objct-str) para float
tt.data['Store']= tt.data['Store'].apply(lambda x:(float(str(x))))
tt.data['Dept']= tt.data['Dept'].apply(lambda x:(float(str(x))))
tt.targed = np.where(tt.targed['IsHoliday']==False, 0, 1)
X_train = tt.data.as_matrix()
Y_train = tt.targed ## Verificar se é necessário transformar tudo em matriz
X_test = tst.data
Y_test = tst.targed

In [ ]:
sns.pairplot(tt[['Store','Dept','IsHoliday','Weekly_Sales']], hue='IsHoliday')

In [ ]:
model = svm.SVC(kernel='poly')
model.fit(X_train, Y_train)

In [ ]:
print(X_train)

In [ ]:
from matplotlib import style
style.use("seaborn-colorblind")
tt.data.plot(x='Dept', y='Weekly_Sales', c=tt.targed, kind='scatter',colormap='Accent_r')

Novamente o departamento 72 apresenta relevância no volume de vendas nos feriados.

In [ ]:
type(X_train)

In [ ]:
## predict case
sns.lmplot('X_test', 'Y_test', data=X_train, hue='Types', palette='Set1' )

In [ ]:
def acuracia(model,X_train,Y_train):
    resultados = cross_val_predict(model,X_train,Y_train, cv=5)
    return metrics.accuracy_score(Y_Train,resultados)
acuracia

Como conclusão podemos aplicar uma maquina de vetores por meio do método polinomial, para prever o volume de vendas do departamento 72 em feriados.  